Wednesday, June 21, 2023



In [1]:
import time
from datetime import date

startTime = time.time()
todaysDate = date.today()

Why do I always have so many issues with spacy??

!python -m spacy download fr  ... this NEVER works!

In [2]:
import spacy
import spacy.cli

In [3]:
spacyDownload = False
if spacyDownload:
    spacy.cli.download("fr_core_news_sm")

In [4]:
# !pip install ecco

In [5]:
endTime = time.time()

elapsedTime = time.strftime("%H:%M:%S", time.gmtime(endTime - startTime))

print(todaysDate.strftime('# Run Date: %A, %B %d, %Y'))
print(f"# Run Time: {elapsedTime}")


# Run Date: Wednesday, June 21, 2023
# Run Time: 00:00:01
